# Основы машинного обучения: лабораторная работа №4
## Деревья решений и ансамблевые методы

В этой лабораторной работе вам предстоит реализовать с нуля алгоритм дерева решений, а затем применить и сравнить различные ансамблевые методы, доступные в библиотеке `scikit-learn`.

### Цель

- Изучить теоретические основы и получить практические навыки реализации деревьев решений.
- Освоить применение ансамблевых методов (бэггинг, бустинг) для решения задачи классификации.
- Научиться сравнивать и анализировать производительность различных моделей машинного обучения.

### Оценивание и баллы

За это задание в общей сложности можно получить **до 10 баллов**. Баллы распределяются по задачам, как описано в ячейках ниже. Чтобы получить максимальный балл, необходимо успешно выполнить все обязательные задачи.

***
### Задачи

#### 1. Определить номер варианта
Перейдите по ссылке из личного кабинета на Google Таблицу со списком студентов. Найдите свое ФИО в списке и запомните соответствующий порядковый номер (поле № п/п) в первом столбце. Заполните его в ячейке ниже и выполните ячейку. Если вы не можете найти себя в списке, обратитесь к своему преподавателю.

In [ ]:
# TODO: Впишите свой номер по списку (STUDENT_ID)
STUDENT_ID = None

Теперь выполните следующую ячейку. Она определит ваш вариант задания и выведет его.

In [ ]:
import pandas as pd

if STUDENT_ID is None:
    print("ОШИБКА! Не указан порядковый номер студента в списке группы.")
else:
    variants = pd.DataFrame([
        {"Dataset": "Прогнозирование оттока клиентов банка", "Dataset URL": "shubhammeshram579/bank-customer-churn-prediction", "Tree Algo": "ID3", "Bagging Algo": "RandomForestClassifier", "Boosting Algo": "AdaBoostClassifier"},
        {"Dataset": "Прогнозирование инсульта", "Dataset URL": "fedesoriano/stroke-prediction-dataset", "Tree Algo": "C4.5", "Bagging Algo": "BaggingClassifier", "Boosting Algo": "GradientBoostingClassifier"},
        {"Dataset": "Качество красного вина", "Dataset URL": "uciml/red-wine-quality-cortez-et-al-2009", "Tree Algo": "ID3", "Bagging Algo": "RandomForestClassifier", "Boosting Algo": "XGBoost"},
        {"Dataset": "Прогнозирование сердечной недостаточности", "Dataset URL": "fedesoriano/heart-failure-prediction", "Tree Algo": "C4.5", "Bagging Algo": "ExtraTreesClassifier", "Boosting Algo": "CatBoost"},
        {"Dataset": "Набор данных о курении", "Dataset URL": "kukuroo3/body-signal-of-smoking", "Tree Algo": "ID3", "Bagging Algo": "BaggingClassifier", "Boosting Algo": "AdaBoostClassifier"},
        {"Dataset": "Удержание клиентов телеком-оператора", "Dataset URL": "blastchar/telco-customer-churn", "Tree Algo": "C4.5", "Bagging Algo": "RandomForestClassifier", "Boosting Algo": "GradientBoostingClassifier"},
        {"Dataset": "Покупка в социальных сетях", "Dataset URL": "rakeshpanigrahi/social-network-ads", "Tree Algo": "ID3", "Bagging Algo": "ExtraTreesClassifier", "Boosting Algo": "XGBoost"},
        {"Dataset": "Оценка риска по кредиту", "Dataset URL": "uciml/german-credit", "Tree Algo": "C4.5", "Bagging Algo": "BaggingClassifier", "Boosting Algo": "CatBoost"},
        {"Dataset": "Прогнозирование диабета", "Dataset URL": "uciml/pima-indians-diabetes-database", "Tree Algo": "ID3", "Bagging Algo": "RandomForestClassifier", "Boosting Algo": "AdaBoostClassifier"},
        {"Dataset": "Обнаружение мошенничества с онлайн-платежами", "Dataset URL": "rupakroy/online-payments-fraud-detection-dataset", "Tree Algo": "C4.5", "Bagging Algo": "BaggingClassifier", "Boosting Algo": "GradientBoostingClassifier"},
    ])
    variant = variants.iloc[STUDENT_ID % len(variants)]
    print(f"Ваш вариант: {STUDENT_ID % len(variants) + 1}")
    print(f"\nДатасет: {variant['Dataset']}")
    print(f"URL для Kaggle API: {variant['Dataset URL']}")
    print(f"\nЧасть 1. Алгоритм дерева для реализации: {variant['Tree Algo']}")
    print(f"Часть 2. Алгоритмы для сравнения:")
    print(f"  - Бэггинг: {variant['Bagging Algo']}")
    print(f"  - Бустинг: {variant['Boosting Algo']}")

#### 2. Загрузка и подготовка данных

Для загрузки датасета из Kaggle рекомендуется использовать Kaggle API. Это избавит вас от необходимости скачивать файлы вручную.

**Инструкция по настройке Kaggle API в Google Colab:**
1.  Зайдите в свой профиль на Kaggle, перейдите в раздел `Account`.
2.  Нажмите на кнопку `Create New API Token`. На ваш компьютер скачается файл `kaggle.json`.
3.  Выполните ячейку с кодом ниже. Она предложит вам загрузить файл. Выберите скачанный `kaggle.json`.
4.  После этого вы сможете скачивать датасеты с помощью команд `!kaggle datasets download ...`.

In [ ]:
from google.colab import files
import os

# Загружаем файл kaggle.json
if not os.path.exists('/root/.kaggle/kaggle.json'):
    uploaded = files.upload()
    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))
    # Создаем папку и перемещаем в нее файл
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("Kaggle API token successfully set up.")
else:
    print("Kaggle API token already exists.")

Теперь, используя URL для Kaggle API из вашего варианта, скачайте и распакуйте датасет. Загрузите данные в DataFrame библиотеки Pandas.

In [ ]:
# TODO: Замените <dataset-url> на URL из вашего варианта
KAGGLE_DATASET_URL = variant['Dataset URL']

!kaggle datasets download -d {KAGGLE_DATASET_URL}

# TODO: Распакуйте скачанный zip-архив (название может отличаться)
# !unzip <archive_name>.zip

# TODO: Загрузите данные в DataFrame
# dataset = pd.read_csv('<file_name>.csv')

#### 3. Анализ и предварительная обработка данных (1 балл)

Прежде чем строить модели, необходимо изучить данные. Проведите базовый анализ:

1.  **Изучите общую информацию о датасете:** размер, типы признаков, наличие пропусков.
2.  **Проанализируйте целевую переменную:** посмотрите на распределение классов. Является ли выборка сбалансированной?
3.  **Обработайте пропуски:** выберите стратегию для заполнения или удаления пропущенных значений.
4.  **Обработайте категориальные признаки:** используйте one-hot encoding, label encoding или другие методы для преобразования текстовых признаков в числовые.
5.  **Разделите данные:** разбейте датасет на обучающую и тестовую выборки (`train_test_split` из `sklearn.model_selection`).
6.  **Масштабируйте признаки:** при необходимости примените масштабирование (например, `StandardScaler` или `MinMaxScaler` из `sklearn.preprocessing`).

В ячейках ниже выполните необходимые шаги и напишите краткие выводы по каждому пункту.

In [ ]:
### BEGIN YOUR CODE

# Общая информация


# Анализ целевой переменной


# Обработка пропусков


# Обработка категориальных признаков


# Разделение на обучающую и тестовую выборки


# Масштабирование признаков (если необходимо)


### END YOUR CODE

**Выводы по анализу и предобработке:**

*Здесь должен быть ваш текст с выводами по каждому шагу предобработки.*

---
### Часть 1: Реализация дерева решений (4 балла)

В этой части вам предстоит реализовать алгоритм построения дерева решений для задачи классификации с нуля, используя только `NumPy`. 

**Критерий разделения:**
-   Если ваш алгоритм - **ID3**, используйте **Information Gain**.
-   Если ваш алгоритм - **C4.5**, используйте **Gain Ratio**.

Вам нужно будет реализовать две основные сущности:
1.  `Node` — узел дерева.
2.  `DecisionTreeClassifier` — сам классификатор.

In [ ]:
import numpy as np

class Node:
    """Класс, представляющий узел в дереве решений."""
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        """
        Args:
            feature (int): Индекс признака для разделения.
            threshold (float): Пороговое значение для разделения.
            left (Node): Левый дочерний узел (для значений <= threshold).
            right (Node): Правый дочерний узел (для значений > threshold).
            value (int): Значение класса (если узел является листом).
        """
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        """Проверяет, является ли узел листовым."""
        return self.value is not None

In [ ]:
# Внимание: нельзя использовать готовые реализации деревьев решений!

class DecisionTreeClassifier:
    """Классификатор на основе дерева решений."""
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None, criterion='id3'):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None
        self.criterion = criterion # 'id3' for Information Gain, 'c4.5' for Gain Ratio

    def fit(self, X, y):
        """Обучает дерево решений."""
        # Убедимся, что X и y - это numpy массивы
        X = np.array(X)
        y = np.array(y)
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        """Делает предсказания для новых данных."""
        X = np.array(X)
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):
        """Рекурсивно строит дерево."""
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Критерии остановки
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # Находим лучшее разделение
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)

        # Разделяем данные
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        """Выбирает лучший признак и порог для разделения."""
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                if self.criterion == 'id3':
                    gain = self._information_gain(y, X_column, threshold)
                elif self.criterion == 'c4.5':
                    gain = self._gain_ratio(y, X_column, threshold)
                else:
                    raise ValueError("Критерий должен быть 'id3' или 'c4.5'")

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold
        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        """Вычисляет Information Gain."""
        # TODO: Реализуйте эту функцию (2 балла)
        ### BEGIN YOUR CODE

        return 0
    
        ### END YOUR CODE

    def _gain_ratio(self, y, X_column, split_thresh):
        """Вычисляет Gain Ratio."""
        # TODO: Реализуйте эту функцию (2 балла)
        ### BEGIN YOUR CODE
        
        return 0
    
        ### END YOUR CODE
    
    def _entropy(self, y):
        """Вычисляет энтропию."""
        # TODO: Реализуйте эту вспомогательную функцию
        ### BEGIN YOUR CODE
        
        return 0
    
        ### END YOUR CODE
        
    def _split_info(self, y, X_column, split_thresh):
        """Вычисляет Split Information для Gain Ratio."""
        # TODO: Реализуйте эту вспомогательную функцию
        ### BEGIN YOUR CODE
        
        return 1 # Возвращаем 1, чтобы избежать деления на ноль, если не реализовано
    
        ### END YOUR CODE

    def _split(self, X_column, split_thresh):
        """Разделяет данные по порогу."""
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        """Проходит по дереву для предсказания одного сэмпла."""
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        """Находит самый частый класс в наборе данных."""
        unique, counts = np.unique(y, return_counts=True)
        return unique[np.argmax(counts)]

Теперь обучите свой классификатор на обучающей выборке и оцените его качество на тестовой. Рассчитайте метрику **Accuracy**.

In [ ]:
from sklearn.metrics import accuracy_score

# TODO: Создайте и обучите экземпляр вашего классификатора
# Установите criterion='id3' или 'c4.5' в зависимости от вашего варианта
### BEGIN YOUR CODE

custom_tree_accuracy = 0

### END YOUR CODE
print(f"Точность (Accuracy) вашего дерева решений: {custom_tree_accuracy:.4f}")

---
### Часть 2: Применение и сравнение ансамблевых методов (3 балла)

Теперь воспользуемся готовыми реализациями из `scikit-learn` для применения ансамблевых методов.

1.  Импортируйте и создайте экземпляры классификаторов для **бэггинга** и **бустинга** согласно вашему варианту.
2.  Обучите обе модели на **обучающей** выборке.
3.  Сделайте предсказания на **тестовой** выборке.
4.  Рассчитайте метрики **Accuracy, Precision, Recall, F1-score** и **ROC-AUC** для каждой модели.
5.  Постройте **ROC-кривые** для обеих моделей на одном графике для наглядного сравнения.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
# Для XGBoost и CatBoost может потребоваться !pip install xgboost catboost
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# TODO: Создайте экземпляры моделей бэггинга и бустинга согласно варианту
### BEGIN YOUR CODE

bagging_model = None # Например: RandomForestClassifier(random_state=42)
boosting_model = None # Например: GradientBoostingClassifier(random_state=42)

# Обучение моделей


# Предсказания


# Расчет метрик


# Построение ROC-кривых


### END YOUR CODE

**Сравнение моделей:**

*Проанализируйте полученные метрики и ROC-кривые. Какая модель показала себя лучше на вашем датасете? Почему, на ваш взгляд? Свяжите свои выводы с особенностями алгоритмов (например, как они работают с выбросами, сложными зависимостями, несбалансированными классами и т.д.).*

---
#### 8. Опишите полученные результаты (1 балл)

Напишите краткие выводы объемом в один абзац, ориентированные на нетехническую аудиторию (например, на вашего менеджера или начальника). Сосредоточьтесь на следующих вопросах:

- Какое из решений (ваше дерево, бэггинг, бустинг) вы бы порекомендовали для решения бизнес-задачи?
- Каковы основные результаты и что они означают на практике (например, "наша модель с точностью 95% определяет потенциально мошеннические транзакции")?
- Какие дальнейшие шаги по улучшению модели вы бы предложили?

**Выводы для нетехнической аудитории:**

*Здесь должен быть ваш текст.*

---
### Нужна помощь?

Если у вас возникли трудности при выполнении задания, попробуйте следующие решения:

- Посмотрите слайды к лекциям по деревьям решений и ансамблевым методам. Слайды можно найти в личном кабинете или в ТГ-канале курса.
- Задайте вопрос преподавателю в ТГ-канале курса.
- Задайте вопрос преподавателю лично в университете.